In [ ]:
import numpy as np
import pandas as pd
import json
import os, sys

import fnmatch


In [ ]:

weights = {'QB': {
        'hgt': [1, 1],
        'spd': [1, 1],
        'endu': [1, 1],
        'thv': [8, 1.5],
        'thp': [4, 1.5],
        'tha': [8, 1.5],
        'bsc': [1, 1],
        'elu': [1, 1],
    },
    'RB': {
        'stre': [4, 1],
        'spd': [8, 1],
        'endu': [1, 1],
        'elu': [8, 2],
        'rtr': [1, 1],
        'hnd': [2, 1],
        'bsc': [4, 1],
        'rbk': [1, 1],
        'pbk': [2, 1],
    },
    'WR': {
        'hgt': [2, 1],
        'stre': [1, 1],
        'spd': [4, 2],
        'endu': [1, 1],
        'elu': [1, 1],
        'rtr': [4, 1],
        'hnd': [8, 1],
        'bsc': [1, 1],
        'rbk': [1, 1],
    },
    'TE': {
        'hgt': [4, 1],
        'stre': [4, 2],
        'spd': [1, 1],
        'endu': [1, 1],
        'elu': [1, 1],
        'rtr': [4, 1],
        'hnd': [4, 1],
        'bsc': [1, 1],
        'rbk': [2, 1],
    },
    'OL': {
        'hgt': [1, 1],
        'stre': [2, 3],
        'spd': [1, 1],
        'rbk': [2, 1],
        'pbk': [2, 1],
    },
    'DL': {
        'hgt': [4, 3],
        'stre': [8, 3],
        'spd': [2, 1],
        'endu': [1, 1],
        'prs': [8, 1],
        'rns': [8, 1],
        'hnd': [1, 1],
    },
    'LB': {
        'hgt': [2, 3],
        'stre': [4, 1],
        'spd': [4, 1],
        'endu': [1, 1],
        'pcv': [2, 1],
        'tck': [8, 1],
        'prs': [2, 1],
        'rns': [2, 1],
        'hnd': [1, 1],
    },
    'CB': {
        'hgt': [1, 1],
        'stre': [1, 1],
        'spd': [8, 2],
        'endu': [1, 1],
        'pcv': [8, 1],
        'rns': [1, 1],
        'hnd': [2, 1],
    },
    'S': {
        'hgt': [1, 1],
        'stre': [3, 1],
        'spd': [4, 1.5],
        'endu': [1, 1],
        'pcv': [4, 1],
        'tck': [4, 1],
        'rns': [2, 1],
        'hnd': [2, 1],
    },
    'K': {
        'kpw': [1, 1],
        'kac': [1, 1],
    },
    'P': {
        'ppw': [1, 1],
        'pac': [1, 1],
    }
}

In [ ]:
df = pd.read_csv('update.csv')
dft = df[['pid','Season']].groupby('pid').min()
dft2 = df[['pid','Season']].groupby('pid').max()

min_season = dict(dict(dft)['Season'])

valid_pids = set(dft[(dft.Season > df.Season.min()) & (dft2.Season < df.Season.max())].index)

# full_careers
df = df[df.pid.isin(valid_pids)]

col_list = list(df.columns)
stats_to_pred = (df.columns[col_list.index('Hgt'):col_list.index('PAc')+1])
from collections import defaultdict
min_stats = {}
year_offsets = defaultdict(list)
for row in df.set_index(['pid','Season'])[stats_to_pred].itertuples():
    #print(row)
    pid = row[0][0]
    year = row[0][1]-min_season[pid]
    ratings = np.array(row[1:])
    if year == 0:
        min_stats[pid] = ratings
    year_offsets[year].append(ratings-min_stats[pid])

In [ ]:
year_off = {k: np.array(v).mean(0) for k,v in year_offsets.items()}

In [ ]:
rosters = []
for file in sorted(os.listdir('.')):
    
    if fnmatch.fnmatch(file, 'fbgm_roster_*.json'):
        print(file)
        #b, ext = os.path.splitext('/path/to/somefile.ext')
        rosters.append(json.load(open(file,'rt')))

In [ ]:
players = []
top_year_list = defaultdict(list)
for roster in rosters:
    year = roster['startingSeason']
    for p in roster['players']:
        name = p['firstName'] + ' ' + p['lastName'] +' '+str(p['draft']['year'])
        name = name.rstrip().strip()
        tot = 0
        for r,w in weights[p['pos']].items():
            tot+=w[0]*p['ratings'][0][r]**w[1]
    
        if p['draft']['year'] >= 2002:
            players.append(p)
            top_year_list[name].append((tot,year,p))

In [ ]:
list(range(10))[2:-2]

In [ ]:
top_year = {}
trimmed_mean =False
for name,results in top_year_list.items():
    sum_rating = defaultdict(float)
    sum_year = 0
    sum_w = 0
    results = sorted(results,key=lambda x:x[0])
    i = len(results)
    # trimmed mean should be unbiased
    if trimmed_mean:
        sk = int(round(0.2*i))
        r2 = results[sk:i-sk]
        if len(r2) > 0:
            results = r2
            #print(sk,i-sk,i,len(results))
        else:
            pass #print('skipped ',sk,i)
    for s,y,r in results:
        if trimmed_mean:
            s = 1
        sum_w += s
        sum_year += y*s
        for k in r['ratings'][0]:
            if k == 'pos':
                continue
            sum_rating[k] += r['ratings'][0][k]*s

    mean_rating = {k:v/sum_w for k,v in sum_rating.items()}
    mean_year = sum_year/sum_w

    tmp = results[0][2]
    for k in tmp['ratings'][0]:
        tmp['ratings'][0][k] = mean_rating.get(k,tmp['ratings'][0][k])
    top_year[name] = (sum_w,int(round(mean_year)),tmp)

In [ ]:
top_year['Adrian Peterson 2007']

In [ ]:
l2 = [_.lower() for _ in stats_to_pred]
l2[1] = 'stre'
l2[3] = 'endu'
l2[-7] = 'tck'

In [ ]:
import copy
for roster in copy.deepcopy(rosters):
    year = roster['startingSeason']

    for _,yr,ratings_o in list(top_year.values()):
        if ratings_o['draft']['year'] > year and yr >= year:
            ratings = copy.deepcopy(ratings_o)
            for o, s in zip(year_off[yr-year],l2):
                ratings['ratings'][0][s] = int(round(np.clip(ratings['ratings'][0][s] + o,0,100)))
            ratings['draft']['year'] = ratings['draft']['year']-1
            ratings['tid'] = -2
            roster['players'].append(ratings)
            #if 'Rodgers' in ratings['lastName']:
            #    print(year,ratings)

    with open('fbgm_draft_{}.json'.format(year),'wt') as fp:
        json.dump(roster,fp, sort_keys=True)

In [ ]:
[_['startingSeason'] for _ in rosters]

In [ ]:
#top_year['Ron Johnson'],top_year['David Carr'],top_year['Julius Peppers'],top_year['Aaron Rodgers']

In [ ]:
[_ for _ in roster['players'] if 'Manziel' in _['lastName']]

In [ ]:
top_year['Johnny Manziel']

In [ ]:
ratings